In [2]:
import pandas as pd
import os 
import numpy as np


In [3]:
print(os.getcwd())


/Users/ruturajdesai/Desktop/Data-Science/projects/end_to_end/flight_end_to_end/flight_prediction/notebooks


In [4]:
os.chdir('../') 

print(os.getcwd())

/Users/ruturajdesai/Desktop/Data-Science/projects/end_to_end/flight_end_to_end/flight_prediction


In [5]:
df=pd.read_csv('./data/raw/flight_price.csv')

In [6]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [16]:
# changing column names
# changing column names
def cleaning(df: pd.DataFrame) -> pd.DataFrame:
    
    return(
        df
        # changing columns to lower case
        .rename(columns=str.lower)
        # renaming columns
        .rename(columns={'source': 'source_city'})
        .rename(columns={'destination': 'destination_city'})
        .rename(columns={'arrival_time': 'arr_time'})
        .rename(columns={'dept_time': 'dep_time'})
        .assign(**{
            col :df[col].str.strip()
            for col in df.select_dtypes(include='O').columns
        })
        # handling price column
        .assign(price=lambda x: x['price'].astype(float))
        # handling total_stops column
        .assign(total_stops=lambda x: x['total_stops'].replace({'non-stop':0, '1 stop':1, '2 stops':2, '3 stops':3, '4 stops':4}))
        # lowering each row value of airline, source_city, destination_city
        .assign(airline=lambda x: x['airline'].str.lower())
        .assign(source_city=lambda x: x['source_city'].str.lower())
        .assign(destination_city=lambda x: x['destination_city'].str.lower())
        .assign(dep_time=lambda x: pd.to_datetime(x['dep_time'],errors='coerce'))
        # extracting dept_hour and dept_min from dept_time
        .assign(dep_hr=lambda x: x['dep_time'].dt.hour)
        .assign(dep_min=lambda x: x['dep_time'].dt.minute)
        .drop(columns=['Airline','Source','Destination','Route','Date_of_Journey', 'Dep_Time',
       'Arrival_Time', 'Duration', 'Total_Stops', 'Additional_Info'])
        #changing values in airline column as it has jet airline business and jet airway so changing jet airway business to jet airway
        .assign(airline =lambda x:x['airline'].replace({'jet airways business':'jet airways',
                                                      'vistara premium economy':'vistara',
                                                      'multiple carriers premium economy':'multiple carriers'}))
        .assign(date_of_journey=lambda x :pd.to_datetime(x['date_of_journey']))
        .assign(arr_time=lambda x : x.loc[:,'arr_time'].str.split().str.get(0))
        #.assign(arr_time=lambda x:pd.to_datetime(x['arr_time']).dt.time)
        .assign(arr_time=lambda x:pd.to_datetime(x['arr_time']))
        .assign(duration=lambda x:
                x['duration'].str.extract(r'(\d+)h')[0].fillna(0).astype(int) * 60 +
                x['duration'].str.extract(r'(\d+)m')[0].fillna(0).astype(int) 
                )

        
        
    )

In [17]:
clean_df=cleaning(df)

/var/folders/bl/yl6vx_451dd0b1qgm5hvh8wh0000gn/T/ipykernel_50234/860923554.py:21: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .assign(total_stops=lambda x: x['total_stops'].replace({'non-stop':0, '1 stop':1, '2 stops':2, '3 stops':3, '4 stops':4}))
/var/folders/bl/yl6vx_451dd0b1qgm5hvh8wh0000gn/T/ipykernel_50234/860923554.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  .assign(dep_time=lambda x: pd.to_datetime(x['dep_time'],errors='coerce'))
/var/folders/bl/yl6vx_451dd0b1qgm5hvh8wh0000gn/T/ipykernel_50234/860923554.py:36: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified

In [18]:
clean_df.head()

,airline,date_of_journey,source_city,destination_city,route,dep_time,arr_time,duration,total_stops,additional_info,price,dep_hr,dep_min
0,indigo,2019-03-24,banglore,new delhi,BLR → DEL,2025-12-04 22:20:00,2025-12-04 01:10:00,170,0.0,No info,3897.0,22,20
1,air india,2019-05-01,kolkata,banglore,CCU → IXR → BBI → BLR,2025-12-04 05:50:00,2025-12-04 13:15:00,445,2.0,No info,7662.0,5,50
2,jet airways,2019-06-09,delhi,cochin,DEL → LKO → BOM → COK,2025-12-04 09:25:00,2025-12-04 04:25:00,1140,2.0,No info,13882.0,9,25
3,indigo,2019-05-12,kolkata,banglore,CCU → NAG → BLR,2025-12-04 18:05:00,2025-12-04 23:30:00,325,1.0,No info,6218.0,18,5
4,indigo,2019-03-01,banglore,new delhi,BLR → NAG → DEL,2025-12-04 16:50:00,2025-12-04 21:35:00,285,1.0,No info,13302.0,16,50


In [19]:
clean_df.head()

,airline,date_of_journey,source_city,destination_city,route,dep_time,arr_time,duration,total_stops,additional_info,price,dep_hr,dep_min
0,indigo,2019-03-24,banglore,new delhi,BLR → DEL,2025-12-04 22:20:00,2025-12-04 01:10:00,170,0.0,No info,3897.0,22,20
1,air india,2019-05-01,kolkata,banglore,CCU → IXR → BBI → BLR,2025-12-04 05:50:00,2025-12-04 13:15:00,445,2.0,No info,7662.0,5,50
2,jet airways,2019-06-09,delhi,cochin,DEL → LKO → BOM → COK,2025-12-04 09:25:00,2025-12-04 04:25:00,1140,2.0,No info,13882.0,9,25
3,indigo,2019-05-12,kolkata,banglore,CCU → NAG → BLR,2025-12-04 18:05:00,2025-12-04 23:30:00,325,1.0,No info,6218.0,18,5
4,indigo,2019-03-01,banglore,new delhi,BLR → NAG → DEL,2025-12-04 16:50:00,2025-12-04 21:35:00,285,1.0,No info,13302.0,16,50


In [11]:
clean_df['Arrival_Time'].unique()

KeyError: 'Arrival_Time'

In [ ]:
# Extract only the time part from arr_time (e.g., '12:30 21 Mar' -> '12:30')
clean_df.assign(arrival_time=lambda x: x['arr_time'].str.strip().str.split(' ').str[0]).head()
#clean_df['arrive_t'] = clean_df['arr_time'].str.strip().str.split(' ').str[0]

AttributeError: Can only use .str accessor with string values!

In [ ]:
pd.reset_option('display.max_rows')

In [20]:
clean_df.head()

,airline,date_of_journey,source_city,destination_city,route,dep_time,arr_time,duration,total_stops,additional_info,price,dep_hr,dep_min
0,indigo,2019-03-24,banglore,new delhi,BLR → DEL,2025-12-04 22:20:00,2025-12-04 01:10:00,170,0.0,No info,3897.0,22,20
1,air india,2019-05-01,kolkata,banglore,CCU → IXR → BBI → BLR,2025-12-04 05:50:00,2025-12-04 13:15:00,445,2.0,No info,7662.0,5,50
2,jet airways,2019-06-09,delhi,cochin,DEL → LKO → BOM → COK,2025-12-04 09:25:00,2025-12-04 04:25:00,1140,2.0,No info,13882.0,9,25
3,indigo,2019-05-12,kolkata,banglore,CCU → NAG → BLR,2025-12-04 18:05:00,2025-12-04 23:30:00,325,1.0,No info,6218.0,18,5
4,indigo,2019-03-01,banglore,new delhi,BLR → NAG → DEL,2025-12-04 16:50:00,2025-12-04 21:35:00,285,1.0,No info,13302.0,16,50


In [21]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   airline           10683 non-null  object        
 1   date_of_journey   10683 non-null  datetime64[ns]
 2   source_city       10683 non-null  object        
 3   destination_city  10683 non-null  object        
 4   route             10682 non-null  object        
 5   dep_time          10683 non-null  datetime64[ns]
 6   arr_time          10683 non-null  datetime64[ns]
 7   duration          10683 non-null  int64         
 8   total_stops       10682 non-null  float64       
 9   additional_info   10683 non-null  object        
 10  price             10683 non-null  float64       
 11  dep_hr            10683 non-null  int32         
 12  dep_min           10683 non-null  int32         
dtypes: datetime64[ns](3), float64(2), int32(2), int64(1), object(5)
memory usage

In [22]:
tf_df=pd.read_csv('./data/interim/cleaned_flight_price.csv')

In [23]:
tf_df.head()

,Unnamed: 0,airline,date_of_journey,source_city,destination_city,route,dep_time,arr_time,duration,total_stops,additional_info,price,dep_hr,dep_min
0,0,indigo,2019-03-24,banglore,new delhi,BLR → DEL,2025-12-03 22:20:00,2025-12-03 01:10:00,170,0.0,No info,3897.0,22,20
1,1,air india,2019-05-01,kolkata,banglore,CCU → IXR → BBI → BLR,2025-12-03 05:50:00,2025-12-03 13:15:00,445,2.0,No info,7662.0,5,50
2,2,jet airways,2019-06-09,delhi,cochin,DEL → LKO → BOM → COK,2025-12-03 09:25:00,2025-12-03 04:25:00,1140,2.0,No info,13882.0,9,25
3,3,indigo,2019-05-12,kolkata,banglore,CCU → NAG → BLR,2025-12-03 18:05:00,2025-12-03 23:30:00,325,1.0,No info,6218.0,18,5
4,4,indigo,2019-03-01,banglore,new delhi,BLR → NAG → DEL,2025-12-03 16:50:00,2025-12-03 21:35:00,285,1.0,No info,13302.0,16,50


In [ ]:
tf_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   airline           10683 non-null  object 
 1   date_of_journey   10683 non-null  object 
 2   source_city       10683 non-null  object 
 3   destination_city  10683 non-null  object 
 4   route             10682 non-null  object 
 5   dep_time          10683 non-null  object 
 6   arr_time          10683 non-null  object 
 7   duration          10683 non-null  int64  
 8   total_stops       10682 non-null  float64
 9   additional_info   10683 non-null  object 
 10  price             10683 non-null  float64
 11  dep_hr            10683 non-null  int64  
 12  dep_min           10683 non-null  int64  
dtypes: float64(2), int64(3), object(8)
memory usage: 1.1+ MB


In [54]:
x_train=pd.read_csv('./data/split/X_train.csv')
y_train=pd.read_csv('./data/split/y_train.csv')
x_val=pd.read_csv('./data/split/X_val.csv')
y_val=pd.read_csv('./data/split/y_val.csv')
x_test=pd.read_csv('./data/split/X_test.csv')
y_test=pd.read_csv('./data/split/y_test.csv')

In [55]:
x_train.head()

,Unnamed: 0,airline,source_city,destination_city,duration,total_stops,additional_info,dep_hr,dep_min,arrival_hr,arrival_min,flight_day,flight_month
0,4535,vistara,kolkata,banglore,695,1.0,No info,7,10,18,45,24,3
1,5420,jet airways,delhi,cochin,810,2.0,In-flight meal not included,5,30,19,0,15,6
2,2983,indigo,banglore,delhi,180,0.0,No info,21,15,0,15,27,4
3,8380,indigo,kolkata,banglore,155,0.0,No info,15,20,17,55,1,3
4,10471,indigo,delhi,cochin,320,1.0,No info,17,5,22,25,15,6


In [56]:
y_train.head()

,price
0,16932.0
1,11507.0
2,3943.0
3,6565.0
4,6147.0


In [57]:
ml_df=x_train.join(y_train).drop(columns=['Unnamed: 0'])

In [58]:
ml_df.head()

,airline,source_city,destination_city,duration,total_stops,additional_info,dep_hr,dep_min,arrival_hr,arrival_min,flight_day,flight_month,price
0,vistara,kolkata,banglore,695,1.0,No info,7,10,18,45,24,3,16932.0
1,jet airways,delhi,cochin,810,2.0,In-flight meal not included,5,30,19,0,15,6,11507.0
2,indigo,banglore,delhi,180,0.0,No info,21,15,0,15,27,4,3943.0
3,indigo,kolkata,banglore,155,0.0,No info,15,20,17,55,1,3,6565.0
4,indigo,delhi,cochin,320,1.0,No info,17,5,22,25,15,6,6147.0


In [59]:
ml_df.describe()

,duration,total_stops,dep_hr,dep_min,arrival_hr,arrival_min,flight_day,flight_month,price
count,6836.000000,6835.000000,6836.000000,6836.000000,6836.000000,6836.000000,6836.000000,6836.000000,6836.000000
mean,642.973230,0.821653,12.512873,24.496050,13.374488,24.707431,13.568607,4.713136,9117.712990
std,505.791404,0.673248,5.727919,18.762127,6.878327,16.528731,8.443372,1.162852,4679.255107
min,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3.000000,1759.000000
25%,170.000000,0.000000,8.000000,5.000000,8.000000,10.000000,6.000000,3.000000,5228.000000
50%,515.000000,1.000000,12.000000,25.000000,14.000000,25.000000,12.000000,5.000000,8372.000000
75%,940.000000,1.000000,18.000000,40.000000,19.000000,35.000000,21.000000,6.000000,12384.000000
max,2820.000000,3.000000,23.000000,55.000000,23.000000,55.000000,27.000000,6.000000,79512.000000


In [60]:
cols=ml_df.columns.tolist()
cols

['airline',
 'source_city',
 'destination_city',
 'duration',
 'total_stops',
 'additional_info',
 'dep_hr',
 'dep_min',
 'arrival_hr',
 'arrival_min',
 'flight_day',
 'flight_month',
 'price']

In [61]:

ml_df[ml_df['price'].isna()]

,airline,source_city,destination_city,duration,total_stops,additional_info,dep_hr,dep_min,arrival_hr,arrival_min,flight_day,flight_month,price


In [62]:
ml_df.drop(index=748,inplace=True)

In [66]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error 
#from xgboost import XGBRegressor
categorical_cols = ['airline', 'source_city', 'destination_city', 'additional_info']
numerical_cols = ['total_stops', 'duration', 'dep_hr', 'dep_min', 'arrival_hr', 'arrival_min', 'flight_day', 'flight_month']
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])
rf=RandomForestRegressor(n_estimators=100, random_state=42)
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model',  rf)  # Add your model here
])
model_pipeline.fit(ml_df.drop(columns=['price']), ml_df['price'])




,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [67]:
model_pipeline.predict(x_val)

array([12702.02,  4161.45, 11447.94, ..., 13025.03,  6304.35, 18390.51],
      shape=(1710,))

In [69]:
print("Model evaluation on validation set:")
val_predictions = model_pipeline.predict(x_val)
print("r2_score:", model_pipeline.score(x_val, y_val))

Model evaluation on validation set:
r2_score: 0.8663762550477145


In [70]:
print("model evaluation on test set:")
test_predictions = model_pipeline.predict(x_test)
print("r2_score:", model_pipeline.score(x_test, y_test))

model evaluation on test set:
r2_score: 0.8723260373511236


In [12]:
transformed_df=pd.read_csv("./data/transformed/transformed_flight_price.csv")
transformed_df.head()

,Unnamed: 0,airline,date_of_journey,source_city,destination_city,duration,total_stops,additional_info,price,dep_hr,dep_min,arrival_hr,arrival_min,flight_day,flight_month,flight_year,day_of_week,is_weekend
0,0,indigo,2019-03-24,banglore,new delhi,170,0.0,No info,3897.0,22,20,1,10,24,3,2019,6,1
1,1,air india,2019-05-01,kolkata,banglore,445,2.0,No info,7662.0,5,50,13,15,1,5,2019,2,0
2,2,jet airways,2019-06-09,delhi,cochin,1140,2.0,No info,13882.0,9,25,4,25,9,6,2019,6,1
3,3,indigo,2019-05-12,kolkata,banglore,325,1.0,No info,6218.0,18,5,23,30,12,5,2019,6,1
4,4,indigo,2019-03-01,banglore,new delhi,285,1.0,No info,13302.0,16,50,21,35,1,3,2019,4,0


In [13]:
transformed_df['flight_year'].unique()

array([2019])

In [14]:
cols=transformed_df.columns.tolist()
for col in cols:
    print(f"col name {col} has {transformed_df[col].isnull().sum()} missing values")

col name Unnamed: 0 has 0 missing values
col name airline has 0 missing values
col name date_of_journey has 0 missing values
col name source_city has 0 missing values
col name destination_city has 0 missing values
col name duration has 0 missing values
col name total_stops has 1 missing values
col name additional_info has 0 missing values
col name price has 0 missing values
col name dep_hr has 0 missing values
col name dep_min has 0 missing values
col name arrival_hr has 0 missing values
col name arrival_min has 0 missing values
col name flight_day has 0 missing values
col name flight_month has 0 missing values
col name flight_year has 0 missing values
col name day_of_week has 0 missing values
col name is_weekend has 0 missing values


In [15]:
transformed_df.describe()

,Unnamed: 0,duration,total_stops,price,dep_hr,dep_min,arrival_hr,arrival_min,flight_day,flight_month,flight_year,day_of_week,is_weekend
count,10683.000000,10683.000000,10682.000000,10683.000000,10683.000000,10683.000000,10683.000000,10683.000000,10683.000000,10683.000000,10683.0,10683.000000,10683.000000
mean,5341.000000,643.093232,0.824190,9087.064121,12.490686,24.411214,13.348778,24.690630,13.508378,4.708602,2019.0,2.934850,0.294299
std,3084.060797,507.862001,0.675229,4611.359167,5.748650,18.767980,6.859125,16.506036,8.479277,1.164357,0.0,2.006698,0.455748
min,0.000000,5.000000,0.000000,1759.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3.000000,2019.0,0.000000,0.000000
25%,2670.500000,170.000000,0.000000,5277.000000,8.000000,5.000000,8.000000,10.000000,6.000000,3.000000,2019.0,1.000000,0.000000
50%,5341.000000,520.000000,1.000000,8372.000000,11.000000,25.000000,14.000000,25.000000,12.000000,5.000000,2019.0,3.000000,0.000000
75%,8011.500000,930.000000,1.000000,12373.000000,18.000000,40.000000,19.000000,35.000000,21.000000,6.000000,2019.0,5.000000,1.000000
max,10682.000000,2860.000000,4.000000,79512.000000,23.000000,55.000000,23.000000,55.000000,27.000000,6.000000,2019.0,6.000000,1.000000


In [18]:
transformed_df[transformed_df['duration'] == transformed_df['duration'].max()]

,Unnamed: 0,airline,date_of_journey,source_city,destination_city,duration,total_stops,additional_info,price,dep_hr,dep_min,arrival_hr,arrival_min,flight_day,flight_month,flight_year,day_of_week,is_weekend
10456,10456,jet airways,2019-03-06,delhi,cochin,2860,2.0,No info,20694.0,19,10,18,50,6,3,2019,2,0


In [21]:
transformed_df.drop(index=10456, inplace=True)



KeyError: '[10456] not found in axis'

In [23]:
transformed_df.describe()


,Unnamed: 0,duration,total_stops,price,dep_hr,dep_min,arrival_hr,arrival_min,flight_day,flight_month,flight_year,day_of_week,is_weekend
count,10682.000000,10682.000000,10681.000000,10682.000000,10682.000000,10682.000000,10682.000000,10682.000000,10682.000000,10682.000000,10682.0,10682.000000,10682.000000
mean,5340.521157,642.885696,0.824080,9085.977532,12.490077,24.412563,13.348343,24.688261,13.509081,4.708762,2019.0,2.934937,0.294327
std,3083.807997,507.432542,0.675164,4610.207146,5.748574,18.768341,6.859299,16.504992,8.479363,1.164294,0.0,2.006771,0.455761
min,0.000000,5.000000,0.000000,1759.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3.000000,2019.0,0.000000,0.000000
25%,2670.250000,170.000000,0.000000,5277.000000,8.000000,5.000000,8.000000,10.000000,6.000000,3.000000,2019.0,1.000000,0.000000
50%,5340.500000,520.000000,1.000000,8372.000000,11.000000,25.000000,14.000000,25.000000,12.000000,5.000000,2019.0,3.000000,0.000000
75%,8010.750000,930.000000,1.000000,12373.000000,18.000000,40.000000,19.000000,35.000000,21.000000,6.000000,2019.0,5.000000,1.000000
max,10682.000000,2820.000000,4.000000,79512.000000,23.000000,55.000000,23.000000,55.000000,27.000000,6.000000,2019.0,6.000000,1.000000


In [32]:
filtered_df = transformed_df[
    transformed_df['duration'].between(
        transformed_df['duration'].quantile(0.9991),
        transformed_df['duration'].quantile(0.9999)
    )
]
filtered_df

,Unnamed: 0,airline,date_of_journey,source_city,destination_city,duration,total_stops,additional_info,price,dep_hr,dep_min,arrival_hr,arrival_min,flight_day,flight_month,flight_year,day_of_week,is_weekend
5701,5701,air india,2019-03-01,banglore,new delhi,2315,2.0,No info,25430.0,8,50,23,25,1,3,2019,4,0
5953,5953,jet airways,2019-03-21,delhi,cochin,2525,2.0,No info,11664.0,10,20,4,25,21,3,2019,3,0
6059,6059,jet airways,2019-03-27,delhi,cochin,2315,2.0,In-flight meal not included,8098.0,5,10,19,45,27,3,2019,2,0
6321,6321,air india,2019-03-01,banglore,new delhi,2345,2.0,No info,17135.0,8,50,23,55,1,3,2019,4,0
6476,6476,jet airways,2019-03-27,delhi,cochin,2315,2.0,In-flight meal not included,8938.0,5,10,19,45,27,3,2019,2,0
7612,7612,air india,2019-03-18,banglore,new delhi,2315,2.0,No info,10783.0,8,50,23,25,18,3,2019,0,0
7906,7906,air india,2019-03-24,kolkata,banglore,2480,2.0,No info,13990.0,5,55,23,15,24,3,2019,6,1
9848,9848,air india,2019-03-01,banglore,new delhi,2345,2.0,No info,17135.0,8,50,23,55,1,3,2019,4,0
10326,10326,air india,2019-03-03,banglore,new delhi,2420,2.0,No info,11791.0,5,50,22,10,3,3,2019,6,1


In [31]:
print(filtered_df.size)

162


In [36]:
filtered_df=filtered_df.assign(duration_hr=lambda x:x['duration']//60)

In [37]:
filtered_df.describe()

,Unnamed: 0,duration,total_stops,price,dep_hr,dep_min,arrival_hr,arrival_min,flight_day,flight_month,flight_year,day_of_week,is_weekend,duration_hr
count,9.000000,9.00000,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.0,9.0,9.000000,9.000000,9.000000
mean,7355.777778,2375.00000,2.0,13884.888889,6.888889,38.333333,19.888889,33.333333,13.666667,3.0,2019.0,3.444444,0.222222,39.222222
std,1718.802910,80.42854,0.0,5377.328669,1.900292,19.039433,6.193635,16.955825,11.884864,0.0,0.0,1.943651,0.440959,1.481366
min,5701.000000,2315.00000,2.0,8098.000000,5.000000,10.000000,4.000000,10.000000,1.000000,3.0,2019.0,0.000000,0.000000,38.000000
25%,6059.000000,2315.00000,2.0,10783.000000,5.000000,20.000000,19.000000,25.000000,1.000000,3.0,2019.0,2.000000,0.000000,38.000000
50%,6476.000000,2345.00000,2.0,11791.000000,8.000000,50.000000,23.000000,25.000000,18.000000,3.0,2019.0,4.000000,0.000000,39.000000
75%,7906.000000,2420.00000,2.0,17135.000000,8.000000,50.000000,23.000000,45.000000,24.000000,3.0,2019.0,4.000000,0.000000,40.000000
max,10326.000000,2525.00000,2.0,25430.000000,10.000000,55.000000,23.000000,55.000000,27.000000,3.0,2019.0,6.000000,1.000000,42.000000
